# Setup
## Downloading dependencies
###### *Cell 1*

In [ ]:
from IPython.display import clear_output
# If it ain't here, you pip it. https://www.w3schools.com/python/python_ref_modules.asp
!pip install --upgrade kaggle
!pip install --upgrade pandas
clear_output()

## Importing dependencies
###### *Cell 2*

In [ ]:
import os
import tkinter as tk
from tkinter import filedialog
import json
from pathlib import Path
import numpy as np
import pandas as pd

## Initialising the Kaggle CLI
### Option #1: New token
If you don't already have a token or have lost the file to your current token, in the settings of your Kaggle account, click on the button 'Generate New Token'. Follow the instructions and copy the alphanumeric string at the top when such a floating dialogue appears. Then, run cell 3 and paste the key when prompted.
###### *Cell 3*

In [ ]:
def kaggle_key() :
    key = input('Paste your token here, or leave empty if your token comes in the form of a JSON file: ')
    clear_output()
    if len(key) > 3 :
        # Create api.txt in working directory, where `KAGGLE_API_TOKEN {key}`.
        api_file = Path("api.txt")
        with api_file.open("w", encoding="utf-8") as f:
            f.write('KAGGLE_API_TOKEN ' + key)
        os.environ['KAGGLE_API_TOKEN'] = key
        os.environ.pop('KAGGLE_USERNAME', None)
        os.environ.pop('KAGGLE_KEY', None)
        print('The last four characters of your API key are: ' + key[-4:] + '. If you suspect that you have entered something wrong, run cell 4 again. Otherwise, you may move to the next section.')
        return True
    else :
        return False
    
def legacy_kaggle_key() :
    '''
    Opens a file dialog, validates the selection, and returns the path 
    if a valid JSON file is selected. Handles all edge cases.
    '''
    root = tk.Tk()
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)
    
    file_path = tk.filedialog.askopenfilename(
        title='Find the file with your Kaggle API key...',
        filetypes=(('JSON files', '*.json'), ('All files', '*.*'))
    )
    
    root.destroy()

    if not file_path :
        # Case: User closes the dialog without selecting anything
        print('No file selected. Run cell 5 again if you like.')
        return
    
    # Now try to open and validate the *contents* of the JSON file
    try :
        with open(file_path, 'r') as f :
            data = json.load(f)
        
        # Case: User selects the correct JSON file that contains their API key
        if 'key' in data and isinstance(data['username'], str) and isinstance(data['key'], str) :
            # Create api.txt in working directory, where `KAGGLE_USERNAME {data['username']}` and `KAGGLE_KEY {data['key']}`.
            api_file = Path("api.txt")
            with api_file.open("w", encoding="utf-8") as f:
                f.write('KAGGLE_USERNAME ' + data['username'] + '\nKAGGLE_KEY ' + data['key'])
            os.environ['KAGGLE_USERNAME'] = data['username']
            os.environ['KAGGLE_KEY'] = data['key']
            os.environ.pop('KAGGLE_API_TOKEN', None)
            print('The last four characters of your API key are: ' + data['key'][-4:] + '. If you suspect that this is not an alphanumeric string, find another file by running cell 5 again. Otherwise, you may move to the next section.')
            
        else :
            # Case: User selects a JSON file, but it's not one that contains their API key
            print('The \'key\' field is missing or invalid. To find another file, run this cell again.')
            
    except Exception :
        print('This file may not contain valid JSON. To find another file, run this cell again.')
        return

# If api.txt is in the working directory, separate by newline, then separate by spaces. Each line has the form '{key} {value}`, where the environment variable `key` should be created with value `value`.
# Otherwise, run `kaggle_key()`. If that returns `False`, then run `legacy_kaggle_key()`.

api_file = Path('api.txt')

if api_file.exists() :
    try :
        with api_file.open('r', encoding='utf-8') as f :
            for line in f :
                line = line.strip()
                if not line or line.startswith('#') :
                    continue

                parts = line.split(None, 1)  # split on first whitespace
                if len(parts) != 2 :
                    continue

                env_key, env_value = parts
                os.environ[env_key] = env_value

    except OSError as e :
        print('Found api.txt but could not read it:\n' + e)
else :
    print('api.txt was not found.')

preview = os.environ.get('KAGGLE_API_TOKEN') or os.environ.get('KAGGLE_KEY')
if preview:
    print('Loaded existing credentials from api.txt. The last four characters of your API key are: ' + preview[-4:] + '. If that looks wrong, run cell 4 or 5.')
else:
    # No api.txt present; go through interactive flow
    if not kaggle_key():
        legacy_kaggle_key()

###### *Cell 4*

In [ ]:
if not kaggle_key() :
    print('You probably didn\'t enter a valid key. Run this cell again if you like.')

###### *Cell 5*

In [ ]:
legacy_kaggle_key()

###### *Cell 6*

In [ ]:
kaggle_module_description = !pip show kaggle
os.environ['PATH'] = os.environ['PATH'] + kaggle_module_description[-3][10:-13] + 'Scripts;'
!kaggle datasets download flkuhm/art-price-dataset -p dataset -f artDataset.csv

I assume:
- Your kernel is running on Python 3.13, and Windows 11.
- You have 'tcl/tk and IDLE' checked this Python environment was installed. In other words, if you were to create and run a cell anywhere in this notebook with the following line `!pip freeze`, you are able to find `tkinter` in the output.
- You are not running the kernel with any virtual environment.
# Preparing the dataset
###### *Cell 7*

In [ ]:
raw_dataset = dataset = pd.read_csv('dataset/artDataset.csv')
raw_dataset

The current columns are:
- `Unnamed: 0`: why even lol
- `price`: Numerical
- `artist`: Categorical
- `title`: Not needed?
- `yearCreation`: Numerical, but can be split into a categorical component
- `signed`: Word frequency
- `condition`: Word frequency
- `period`: Categorical, but might line up with `yearCreation`.
- `movement`: Categorical or word frequency

## `Unnamed: 0`
At first glance, the values in this column line up with the values as prescribed by the leftmost index column. There are a variety of parameters when using `pandas.DataFrame.to_csv()` to save a pandas DataFrame to `.csv`. If there is at least one column filled entirely with unique, non-empty values, `index_label` can be used to designate one of them as the index column. Otherwise, `index` can be used to influence whether a new column of indices is created. If an index-like column already exists in the DataFrame, but isn't designated as such, pandas will treat it like any other column, as it could contain real information. As the `Unnamed: 0` column seems to be a common enough phenomenon within datasets uploaded to Kaggle (https://www.kaggle.com/discussions/general/354943), I believe this is what happened in the creation of this dataset.

To find out if `Unnamed: 0` is effectively an index column, I created a filter to find any rows whose `Unnamed: 0` value is different from the index column's.
###### *Cell 8*

In [ ]:
dataset[dataset['Unnamed: 0'] != dataset.index]

There were none. Therefore, all rows had values which corresponded with their indices. Since `Unnamed: 0` is an index column and does not contain any other unique information, I chose to remove it.
###### *Cell 9*

In [ ]:
dataset = dataset.drop(columns = ['Unnamed: 0'])

## `price`
The prices in each row seem to follow a pattern. The United States dollar (USD) is specified as a currency, and the thousands are separated by periods (`.`). This is reminiscent of a convention of writing large numbers that is popular in continental Europe. In contrast to the Anglophone convention, which uses commas (`,`) to separate thousands, and a single period to mark the beginning of the decimal part, the continental convention swaps them around. Nevertheless, characters like these, including the `USD` suffix, cause the prices to be represented as strings. I will need to convert them into numerical representations so that I can carry out sorting and linear regression.

I sought to verify these assumptions:
1. That all prices are denominated in USD.
2. That the period is used as a thousands separator at all times.

First, I searched for any rows whose price was not annnotated with `' USD'`.
###### *Cell 10*

In [ ]:
dataset[~dataset['price'].str.endswith(' USD')]

There were none, meaning that all prices were annotated in `' USD'`. There could be additional clerical errors, like `'USD'` appearing more than once. To test that, I provisionally removed the `' USD'` suffix in a copy of the column, then tested for the presence of additional letters by combining all the strings into one and listing all the unique characters within that string.
###### *Cell 11*

In [ ]:
column_price_numerical = dataset['price'].str.replace(' USD', '')
set(column_price_numerical.str.cat())

The removal of `' USD'` was a success, and caused all the strings to no longer contain any letters. This proved to me that all prices are indeed denominated in USD, removing the need to perform currency conversions. Furthermore, the period is the only symbol to appear, with whitespaces (` `) and commas notably absent.

I could use this uniformity in formatting to my advantage, and further assume that all prices are natural numbers, i.e., none of the prices have decimal parts. I still considered the edge case that the period is used both as a thousands separator and a decimal separator, and to disprove that I checked the prices against the following regular expression `^\d{1,3}(\.\d{3})*$`. Here are some characteristics:
- Without a period, there can only be up to 3 free digits. With periods, there can only be up to 3 leading digits before the leftmost period.
- There must be at least 1 free digit. Without a period, this ensures the number has at least 1 digit. With periods, this ensures the leftmost period is not exposed (`0.100` as opposed to `.100`).
- Every period must be succeeded by exactly 3 digits. There can be as many groups as possible of periods and 3 trailing digits to represent the powers of a thousand, like millions and billions.
- This expression specifically disallows the common convention of denoting cents with a period and 2 trailing digits.

###### *Cell 12*

In [ ]:
column_price_numerical.str.fullmatch(r'^\d{1,3}(\.\d{3})*$').all()

Since all prices fit the above pattern, I'm sufficiently convinced that all prices are to the nearest dollar, and that the period is only ever used as a thousands separator. I acknowledge the deeper edge case that prices are shown to three decimal places (i.e., thousandths of a US dollar). However, apart from being highly unlikely that prices are denominated in anything other than dollars and cents, the smallest division of the US dollar is the cent, which is a hundredth of a dollar. Should the agreed-upon price contain part of a cent, buyers may find it hard to pay the exact amount. Still, in case of any further doubt, you may manually review the dataset on Kaggle or a spreadsheet viewer of your choice. You may find and remove any entry whose price you feel should be reasonably interpreted as being a thousandth of a dollar, and run this section again.

Since I assumed the period is not a decimal separator, I assumed too that removing periods will not cause the price to be interpreted as 100 times larger. Thus, I removed all the periods and parsed all strings as integers, which completed the preprocessing of this column. The prices could now be sorted and used in regression, as seen below.
###### *Cell 13*

In [ ]:
dataset['price'] = pd.to_numeric(column_price_numerical.str.replace('.', ''))
dataset.sort_values(by = ['price'])

## `artist`
Some artists are more famous than others, and consistently command higher prices. Therefore, the provenance of a work should be a predictor of its price. To model this, I sought to turn this column into two features:
  - Dummy (one-hot) encodings for prolific artists. The result is as many columns of dummy encodings as there are artists I wish to track. Any work would take a value of `1` on the column that represents its artist, and `0` everywhere else. The names for each of these columns shall be copied from the strings in `artist`.
  - `works_by_artist`: The number of works in this dataset that are from the same artist. I hypothesised that even if a buyer was unfamiliar with a particular artist, the fact that an auction house's catalogue features many works from the same artist could give an impression of relevance, popularity or market confidence. The result is a numerical column.

My main consideration was the variability introduced during data entry. Different works may have been catalogued decades apart, by different appraisers, who may or may not follow the prevailing formatting guides of their time. Auction houses somewhat mitigate this through authority control, using internal databases to reconcile pseudonyms, name changes (e.g., after marriage), and other variants of a creator’s identity. However, mismatches still arise, and many must be resolved by the curators'/buyers' understanding. For example, if a system doesn't have the right heuristics to link two identities together, it may consider two works, attributed to "PICASSO" and "Pablo Picasso" respectively, to be unrelated. It is up to the buyer to make that connection between the two names, and correctly identify that either of these works are just as valuable as the other.

Considering that this dataset may contain legacy formatting and clerical inconsistencies, and considering that a heuristic approach would not fit within the amount of effort I expect in this section, I decided on human judgement as the most practical choice in consolidating artist names. So, I performed a manual pass to merge entries that were clearly intended to refer to the same person. First, I needed to grasp the space of possible data I needed to process.
###### *Cell 14*

In [ ]:
def tablify(array, columns, transpose = False) :
    rows = int(np.ceil(len(array) / columns))
    array_copy = array + [''] * (rows * columns - len(array))

    if transpose :
        array_as_table = np.array(array_copy).reshape(columns, rows).transpose()
    else :
        array_as_table = np.array(array_copy).reshape(rows, columns)
    print(pd.DataFrame(array_as_table).to_string(index = False, header = False))

artist_names = dataset['artist'].value_counts(dropna = False).sort_index().index.to_list()
tablify(artist_names, 5, True)

I created a helper function that displays long lists in a tabular format, and passed in the unique artist names alphabetically. This sped up my discovery of potential clerical inconsistencies. In many cases, small formatting variations such as differences in spacing, punctuation, or capitalisation become obvious when the entries are viewed side-by-side. For example, I could immediately see on the rightmost column that "T.L. Solien" appeared just below "T. L. Solien", as well as another variant a little further up: "Solien T.L.".

Listed last in the alphabetical ordering, was `nan`.
###### *Cell 15*

In [ ]:
dataset[dataset['artist'].isna()]

There was surprisingly only one `nan` value in this entire dataset. However, this lone `nan` caused friction with many of the pandas functions, and I had to include extra parameters in my function calls, like `dropna = False`. So I replaced it with the string "unknown". This choice avoided conflicts with real artist names, and retained its convenient position at the bottom of alphabetical listings.
###### *Cell 16*

In [ ]:
dataset.loc[725, 'artist'] = 'unknown'

With the full set of unique names in alphabetical order, I began manually identifying connections between these names—an example of data linkage, as I later learnt the process is called.

ChatGPT was useful in accelerating the discovery of possible matches. As someone unfamiliar with all of the artists in the dataset, it brought to the table contextual information, such as an artist's history, aliases, or typical subject matter. With this knowledge, it could propose matches that I wouldn't have otherwise considered. However, given the length of the list of names, it hallucinated often, made up new names and proposed links between unrelated artists. Therefore, I used these suggestions as prompts for further verification rather than as authoritative claims.
###### *Cell 17*

In [ ]:
dataset[dataset['artist'].str.contains('Kota')]

One such case where I overruled ChatGPT was with [Kota Ezawa](https://en.wikipedia.org/wiki/Kota_Ezawa). He is a German and Japanese artist currently based in San Francisco, whose works remix existing media (like photographs and movie stills) in his style of flat colours and a limited palette. As a result, the titles of his works contain credits to the source material, in the form of "(After)".

<table>
    <tr>
        <td>
            <img src="https://sothebys-md.brightspotcdn.com/72/8a/8076c8744baeafc979cf7d160c85/bsp7f-front.png" />
        </td>
        <td>
            <img src="https://sothebys-md.brightspotcdn.com/e3/21/b2d2428f4169b020afa671e01d5f/bsp7g-front.png" />
        </td>
    </tr>
    <tr>
        <td colspan="2"> <!-- Use colspan to span across 2 columns -->
            <p><center><i>"Conical Intersect" (left) and "The Bohemians" (right). Images provided by Sotheby's online catalogue.</i></center></p>
        </td>
    </tr>
</table>

ChatGPT suggested that "The Bohemians (After August Sander 1924)", attributed to "After Kota Ezawa", was the work of an unnamed artist, who was reinterpreting Ezawa’s own interpretation of August Sander. However, I considered this unlikely. If The Bohemians had been made by someone else, I would expect that it was presented in a different condition than the other genuine Ezawas, be offered at a different price, or was at least dated later than them. But in all these respects it was similar to the other Ezawas. Most of the descriptive fields had a consistent format, if not outright identical, and appeared in adjacent index positions. This implied to me that they were catalogued at the same time and by the same appraiser.

Still, I verified my assumptions with a manual search. Although the work has been made unavailable today, I found its [listing](https://www.sothebys.com/en/buy/_the-bohemians-after-august-sander-1924-from-the-history-of-photography-remix-3eed). Here, I saw that not only were the style and colour palette similar to the other Ezawas, the description, which is missing from the dataset as a field, also read:
> Kota Ezawa (German, b. 1969).
>
> This piece is final sale and not eligible for return.

With this, I concluded that ChatGPT was wrong in this instance, and that The Bohemians was indeed created by Kota Ezawa.

After identifying variants of a single artist's name, the next step was to make one of them the *canonical form*: a single identity with which all other variants shall be reconciled. All works in the dataset that had been attributed to any of these variants were re-attributed to this canonical form. Typically, I chose the identity with the most amount of works to be the canonical form.
###### *Cell 18*

In [ ]:
# Merge variants of "T.L. Solien"
dataset['artist'] = dataset['artist'].replace(['T. L. Solien', 'Solien T.L.'], 'T.L. Solien')

# Merge variants of "Kota Ezawa"
dataset['artist'] = dataset['artist'].replace('After Kota Ezawa', 'Kota Ezawa')

# Merge variants of "John Andre Gundelfinger"
dataset['artist'] = dataset['artist'].replace('John Gundelfinger', 'John Andre Gundelfinger')

# Merge variants of "Richard Woods"
dataset['artist'] = dataset['artist'].replace('RICHARD WOODS', 'Richard Woods')

As I do not have the expertise of an art curator, I was necessarily conservative in discovering links between the various identities. Beyond the clerical variants, I did not make further assumptions about any other pair of names, and it is possible that some pseudonyms still lie undiscovered.

Now that the works have been (sufficiently) correctly attributed, some artists now have more works to their name, which more accurately reflects their perceived popularity in the online gallery. I computed this number for each artist and added them to each of their works as a numerical feature: `works_by_artist`, as I had introduced in the beginning of this section.

Notably, I assigned a value of 0 to the work with the unknown artist. While it is certain that this work was created by someone, the absence of a traceable or recognisable identity removes any potential authorship premium. From a buyer’s perspective, unknown authorship introduces uncertainty and eliminates the possibility of prior recognition, institutional validation, or reputational signalling. Even artists represented by a single work retain a non-zero probability of name recognition, whereas an unknown artist does not. For this reason, I treat the unknown artist as having zero name-based recognition. Any value attributed to this work must therefore arise from other observable characteristics, such as aesthetic qualities, condition, or alignment with prevailing trends.
###### *Cell 19*

In [ ]:
artist_popularity = dataset['artist'].value_counts()
dataset.insert(loc = 2, column = 'works_by_artist', value = dataset['artist'].map(artist_popularity))
dataset.loc[725, 'works_by_artist'] = 0
dataset

# Let's take a break for the sponsor of this notebook:
![](https://attachments-cdn-s.coub.com/coub_storage/coub/simple/cw_image/6f66b8f55da/2a6042a632c0744d1e9fe/1448960136_00030.jpg)  
While `works_by_artist` sought to capture the effect of confidence derived from seeing many works from the same artist, individual artists carry their own reputations too. Just knowing that a work was created by a renowned artist can add to its perceived value. I sought to encode this aforementioned "authorship premium" by adding a column for each artist, denoting whether a work was attributed to them. This allows a model to learn the "value" of an artist's name, independent of the qualities and aesthetics of their works.

However, as it was seen in the preview of the dataset above, and as seen below, the majority of artists appear only once: 318 out of 449.
###### *Cell 20*

In [ ]:
artist_popularity = artist_popularity.drop('unknown')
artist_attribution_thresholds = artist_popularity.value_counts().sort_index(ascending = False)
artist_attribution_thresholds

Dummy variables are extremely sparse; a column corresponding to an artist with a single work would take the value 1 for exactly one row and 0 everywhere else. As such a feature carries no repeatable signal, its coefficient would be estimated from a single observation and would therefore primarily capture noise rather than a stable effect attributable to the artist’s reputation. Including hundreds of such sparse features would greatly increase the dimensionality of the dataset without providing meaningful predictive power, increasing the risk of overfitting. At the same time, I consider artist indicators to act as tags rather than numerical features. I'm not measuring the *Pavel Tchelitchew*-ness of a work just like I am the price or year of creation. Instead, if a work had been made by Pavel Tchelitchew, the value associated with the tag is added to its predicted price, like a bias. Under this reasoning, one-work artists account for 318 of the 754 works (42%), and excluding these tags from being learned would remove any possibility of modelling name-based effects for a large subset of works.

In the end, I decided to track every single artist (apart from "unknown") in a separate DataFrame.
###### *Cell 21*

In [ ]:
artist_attributions = pd.DataFrame(
    {
        artist: (dataset['artist'] == artist).astype(int)
        for artist in artist_popularity.to_dict()
    },
    index=dataset.index
)
artist_attributions

This completes the first set of features that I had introduced in the beginning of the section: dummy encodings for prolific artists. `artist_attributions` is designed to be modular; the columns are ordered in descending artist representation, and leftmost columns represent artists who have the most works. Rather than permanently committing to a fixed subset of artist indicators, a selected subset of its columns will be appended to the dataset. `artist_attribution_thresholds`, as shown in cell 20, is a reference for how many of the columns to include for training. It shows that the first 3 columns represent artists who have 13 or more works. The next 19 columns represented artists who have 5 or more works, and so on. By progressively including more columns and therefore less represented artists, I hoped to find the balance between the comprehensive tracking of artists' values and the noisiness of the resulting artist tags.

## `title`
I acknowledge that the title of an artwork influences a prospective buyer. First impressions matter in a marketplace of many works, and an engaging title would help a particular piece stand out to them, rather than be glossed over. However, most titles are unique to a single work and there is no obvious ordinal or numerical structure. To use title as an input to a linear regression model would require transforming the text into a set of bag-of-words features, which, due to the small sample size and large word space, would result in high sparsity and greatly increase the risk of overfitting. Therefore, I decided to retain `title` for my reference, but exclude it from the set of predictive features.

I created a filter to exclude these rows.

## `yearCreation`

In [ ]:
year_conversion = dataset['yearCreation'].value_counts(dropna = False).to_frame()
year_conversion

In [ ]:
def rule_numerical_year(year) :
    if year.isnumeric() :
        return int(year) + 0.5
    return None

def rule_numerical_year_interval(year) :
    if year.isnumeric() :
        return 1
    return None

year_conversion['year'] = year_conversion.index.map(rule_numerical_year)
year_conversion['year_interval'] = year_conversion.index.map(rule_numerical_year_interval)

unconverted_strings = year_conversion['year'].isna()
year_conversion[~unconverted_strings]

In [ ]:
def rule_circa_year(year) :
    if not year.startswith('Circa '):
        return None
    year = year[6:]
    if year.isnumeric():
        return int(year)
    return None

def rule_circa_year_interval(year) :
    if not year.startswith('Circa '):
        return None
    year = year[6:]
    if year.isnumeric():
        return 5
    return None

year_conversion.loc[unconverted_strings, 'year'] = year_conversion.loc[unconverted_strings].index.map(rule_circa_year)
year_conversion.loc[unconverted_strings, 'year_interval'] = year_conversion.loc[unconverted_strings].index.map(rule_circa_year_interval)

unconverted_strings = year_conversion['year'].isna()
year_conversion[unconverted_strings]

In [ ]:
def rule_year_range(year) :
    if year.startswith('Circa '):
        year = year[6:]
    start_end = year.replace(' ', '').split('-')
    if not len(start_end) == 2 :
        return None
    if not (start_end[0].isnumeric() and start_end[1].isnumeric()) :
        return None
    return (int(start_end[0]) + int(start_end[1])) / 2 + 0.5

def rule_year_range_interval(year) :
    if year.startswith('Circa '):
        year = year[6:]
    start_end = year.replace(' ', '').split('-')
    if not len(start_end) == 2 :
        return None
    if not (start_end[0].isnumeric() and start_end[1].isnumeric()) :
        return None
    return int(start_end[1]) - int(start_end[0]) + 1

year_conversion.loc[unconverted_strings, 'year'] = year_conversion.loc[unconverted_strings].index.map(rule_year_range)
year_conversion.loc[unconverted_strings, 'year_interval'] = year_conversion.loc[unconverted_strings].index.map(rule_year_range_interval)

unconverted_strings = year_conversion['year'].isna()
year_conversion[unconverted_strings].index

In [ ]:
dataset[dataset['yearCreation'] == '1998 / 2011']

In [ ]:
year_conversion.loc['Second Half 20th Century ', ['year', 'year_interval']] = [1975, 50]
year_conversion.loc['21st Century ', ['year', 'year_interval']] = [2011.11, 22.22]
year_conversion.loc['2022', ['year', 'year_interval']] = [2022.11, 0.22] # By this logic, "2022" should also only span from January the 1st to March the 20th.
year_conversion.loc['Late 20th Century ', ['year', 'year_interval']] = [1983 + 1 / 3, 100 / 3]
year_conversion.loc['Mid 20th Century ', ['year', 'year_interval']] = [1950, 100 / 3]
year_conversion.loc['Late 19th Century ', ['year', 'year_interval']] = [1883 + 1 / 3, 100 / 3]
year_conversion.loc['Early 20th Century ', ['year', 'year_interval']] = [1916 + 2 / 3, 100 / 3]
year_conversion.loc['First Half 20th Century ', ['year', 'year_interval']] = [1925, 50]
year_conversion.loc['19th Century ', ['year', 'year_interval']] = [1850, 100]
year_conversion.loc['20th Century ', ['year', 'year_interval']] = [1950, 100]
year_conversion.loc['Second Half 19th Century ', ['year', 'year_interval']] = [1875, 50]
year_conversion.loc['1961, printed in 2010', ['year', 'year_interval']] = [2010.5, 1]
year_conversion.loc[year_conversion.index.str.contains('3D'), ['year', 'year_interval']] = [2019.5, 1]
year_conversion.loc['1998 / 2011', ['year', 'year_interval']] = [2011.5, 1]
year_conversion.loc['Printed 1984', ['year', 'year_interval']] = [1984.5, 1]
year_conversion.loc['[nan]', ['year', 'year_interval']] = [1911.11, 222.22] # January the 1st, 1800, to Match the 20th, 2022.

year_conversion[year_conversion['year'].isna()]

In [ ]:
dataset.insert(4, 'year', dataset['yearCreation'].map(year_conversion['year']))
dataset.insert(5, 'year_interval', dataset['yearCreation'].map(year_conversion['year_interval']))
dataset = dataset.drop(columns = ['yearCreation'])
dataset

In [ ]:
dataset['period'].value_counts()

In [ ]:
dataset[(dataset['period'] == '[nan]') & (dataset['year'] != 1911.11)].sort_values(['year'])

In [ ]:
dataset[(dataset['period'] == '19th Century') & (dataset['year'] != 1911.11)].sort_values(['year'])

# Visualisations
Mean mode median? Spread skewness
## Correlation between `period` and `year`